# Sample ARC Submission

This is a sample notebook that can help get you started with creating an ARC Prize submission. It covers the basics of loading libraries, loading data, implementing an approach, and submitting.

You should be able to submit this notebook to the evaluation portal and have it run successfully (although you'll get a score of 0, so you'll need to do some work if you want to do better!)


# Load needed libraries

Basic libraries like numpy, torch, matplotlib, and tqdm are already installed.

In [1]:
import json
from tqdm import tqdm

In [2]:
from random import choice

# Load the data

Here we are loading the training challenges and solutions (this is the public training set), the evaluation challenges and solutions (this is the public evaluation set), and the test challenges (currently a placeholder file that is a copy of the public evaluation challanges).

For your initial testing and exploration, I'd recommend not using the public evaluation set, just work off the public training set and then test against the test challenges (which is actually the public evaluation set). However, when competing in the competition, then you can should probably use the evaluation tasks for training too.

In [3]:
# Public training set
train_challenges_path = '../input/arc-prize-2024/arc-agi_training_challenges.json'
train_solutions_path = '../input/arc-prize-2024/arc-agi_training_solutions.json'

with open(train_challenges_path) as fp:
    train_challenges = json.load(fp)
with open(train_solutions_path) as fp:
    train_solutions = json.load(fp)

# Public evaluation set
evaluation_challenges_path = '../input/arc-prize-2024/arc-agi_training_challenges.json'
evaluation_solutions_path = '../input/arc-prize-2024/arc-agi_training_solutions.json'

with open(evaluation_challenges_path) as fp:
    evaluation_challenges = json.load(fp)
with open(evaluation_solutions_path) as fp:
    evaluation_solutions = json.load(fp)

# This will be the hidden test challenges (currently has a placeholder to the evaluation set)
test_challenges_path = '../input/arc-prize-2024/arc-agi_test_challenges.json'

with open(test_challenges_path) as fp:
    test_challenges = json.load(fp)

Here is an example of what a test task looks like:

In [4]:
sample_task = list(test_challenges.keys())[0]
challenge_ex = test_challenges[sample_task]
challenge_ex
# sample_task

{'test': [{'input': [[3, 2], [7, 8]]}],
 'train': [{'input': [[8, 6], [6, 4]],
   'output': [[8, 6, 8, 6, 8, 6],
    [6, 4, 6, 4, 6, 4],
    [6, 8, 6, 8, 6, 8],
    [4, 6, 4, 6, 4, 6],
    [8, 6, 8, 6, 8, 6],
    [6, 4, 6, 4, 6, 4]]},
  {'input': [[7, 9], [4, 3]],
   'output': [[7, 9, 7, 9, 7, 9],
    [4, 3, 4, 3, 4, 3],
    [9, 7, 9, 7, 9, 7],
    [3, 4, 3, 4, 3, 4],
    [7, 9, 7, 9, 7, 9],
    [4, 3, 4, 3, 4, 3]]}]}

# Generating a submission

To generate a submission you need to output a file called `submission.json` that has the following format:

```
{"00576224": [{"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]}],
 "009d5c81": [{"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]}],
 "12997ef3": [{"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]},
              {"attempt_1": [[0, 0], [0, 0]], "attempt_2": [[0, 0], [0, 0]]}],
 ...
}
```

In this case, the task ids come from `test_challenges`. There may be multiple (i.e., >1) test items per task. Therefore, the dictionary has a list of dicts for each task. These submission dictionaries should appear in the same order as the test items from `test_challenges`. Additionally, you can provide two attempts for each test item. In fact, you **MUST** provide two attempts. If you only want to generate a single attempt, then just submit the same answer for both attempts (or submit an empty submission like the ones shown in the example snippit just above).

In [5]:
from arc_bfs import arc_search, execute_transformation

In [6]:
# Create an empty submission dict for output
submission = {}

# iterate over the test items and build up submission answers
count = 0
for key, task in tqdm(test_challenges.items()):

    # Here are the task's training inputs and outputs
    train_inputs = [item['input'] for item in task['train']]
    train_outputs = [item['output'] for item in task['train']]
    
    test_inputs = [item['input'] for item in task['test']]

    # Here we generate outputs for each test item.
    submission[key] = []

    transformations = set()
    for input_grid, output_grid in zip(train_inputs, train_outputs):
        transformations.add(tuple(arc_search(input_grid, output_grid, max_depth=2)))
    
    transformations = sorted(list(transformations), key=len, reverse=True) # longest solutions come first to avoid empty lists

    for input_grid in test_inputs:
        attempt_dict = dict()
        for i in range(2):
            trans = transformations[i % len(transformations)]
            soln = execute_transformation(input_grid, trans)
            attempt_dict[f'attempt_{i+1}'] = soln
            # if soln != input_grid:
            #     print("CHANGED SOMETHING!!")
        submission[key].append(attempt_dict)
            
# Here we write the submissions to file, so that they will get evaluated
with open('submission.json', 'w') as fp:
    json.dump(submission, fp)

100%|██████████| 400/400 [00:36<00:00, 10.95it/s]


Here is what our submission for the test task above looks like:

In [7]:
submission[sample_task]

[{'attempt_1': [[3, 2], [7, 8]], 'attempt_2': [[3, 2], [7, 8]]}]

# Scoring Your Submission

If you do not want to wait for gradescope to score your solution, we have provided the following code to score your submission. Note that the maximum possibe score is 400.

In [8]:
def score_submission():
    with open('../input/arc-prize-2024/arc-agi_evaluation_solutions.json', 'r') as sol_file:
        solutions = json.load(sol_file)
    
    with open('submission.json', 'r') as sub_file:
        submission = json.load(sub_file)
    
    overall_score = 0

    for task in solutions:
        score = 0
        for i, answer in enumerate(solutions[task]):
            attempt1_correct = submission[task][i]['attempt_1'] == answer
            attempt2_correct = submission[task][i]['attempt_2'] == answer
            score += int(attempt1_correct or attempt2_correct)

        score /= len(solutions[task])

        overall_score += score
    

    print(overall_score)

You can run the above code by uncommenting the following code block. 

In [9]:
score_submission()

3.5


# Confused about where to get started? 

If you're not sure what an initial solution might look like, then consider looking at public notebooks here: https://www.kaggle.com/competitions/arc-prize-2024/code or joining the public discussion here: https://www.kaggle.com/competitions/arc-prize-2024/discussion.

One example notebook that uses a very simple knowledge-based approach is this one: https://www.kaggle.com/code/michaelhodel/program-synthesis-starter-notebook/notebook, which conducts search over a space of domain specific block languages to form hypotheses and then applies these to test items.